<a href="https://colab.research.google.com/github/Hemen-G/Mango-leaf-disease-detection/blob/main/Mango_Disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#connection with drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split


SOURCE_DIR = "/content/drive/MyDrive/MangoLeafBD/MangoLeafBD Dataset"


TARGET_DIR = "/content/drive/MyDrive/dataset/mango"

splits = ['train', 'validation', 'test']


for split in splits:
    for class_name in os.listdir(SOURCE_DIR):
        os.makedirs(os.path.join(TARGET_DIR, split, class_name), exist_ok=True)


for class_name in os.listdir(SOURCE_DIR):
    class_dir = os.path.join(SOURCE_DIR, class_name)
    images = os.listdir(class_dir)


    images = [img for img in images if img.lower().endswith(('.jpg', '.jpeg', '.png'))]


    train_imgs, temp_imgs = train_test_split(images, test_size=0.3, random_state=42)

    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=1/3, random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(class_dir, img), os.path.join(TARGET_DIR, 'train', class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(class_dir, img), os.path.join(TARGET_DIR, 'validation', class_name, img))
    for img in test_imgs:
        shutil.copy(os.path.join(class_dir, img), os.path.join(TARGET_DIR, 'test', class_name, img))

print("✅ Dataset successfully split into train (70%)/validation (20%)/test (10%) sets!")

✅ Dataset successfully split into train (70%)/validation (20%)/test (10%) sets!


In [ ]:
import os


TARGET_DIR = "/content/drive/MyDrive/dataset/mango"


splits = ['train', 'validation', 'test']

for class_name in os.listdir(os.path.join(TARGET_DIR, 'train')):
    print(f"\nClass: {class_name}")
    total_count = 0
    counts = {}

    for split in splits:
        split_path = os.path.join(TARGET_DIR, split, class_name)
        if os.path.exists(split_path):
            count = len(os.listdir(split_path))
            counts[split] = count
            total_count += count


    for split in splits:
        if split in counts:
            percentage = (counts[split] / total_count) * 100
            print(f"{split.capitalize()}: {counts[split]} images ({percentage:.1f}%)")

    print(f"Total: {total_count} images")


print("\nOverall Distribution:")
total_images = 0
split_counts = {'train': 0, 'validation': 0, 'test': 0}

for split in splits:
    split_dir = os.path.join(TARGET_DIR, split)
    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        count = len(os.listdir(class_dir))
        split_counts[split] += count
        total_images += count

for split, count in split_counts.items():
    percentage = (count / total_images) * 100
    print(f"{split.capitalize()}: {count} images ({percentage:.1f}%)")

print(f"Total images in dataset: {total_images}")


Class: Anthracnose
Train: 350 images (70.0%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 500 images

Class: Bacterial Canker
Train: 349 images (69.9%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 499 images

Class: Cutting Weevil
Train: 350 images (70.0%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 500 images

Class: Die Back
Train: 350 images (70.0%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 500 images

Class: Gall Midge
Train: 350 images (70.0%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 500 images

Class: Healthy
Train: 350 images (70.0%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 500 images

Class: Powdery Mildew
Train: 350 images (70.0%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 500 images

Class: Sooty Mould
Train: 349 images (69.9%)
Validation: 100 images (20.0%)
Test: 50 images (10.0%)
Total: 499 images

Overall Distribution:
Train: 2798 images (70

#importing Libraries


In [2]:
!pip install tensorflow matplotlib pillow scikit-learn seaborn pandas

In [3]:


# Import everything
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from PIL import Image
import os
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight

print("✅ All packages imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print("GPU Available:", tf.config.list_physical_devices('GPU'))

✅ All packages imported successfully!
TensorFlow version: 2.19.0
GPU Available: []


#Data preprocessing

###Training Image Preprocessing

In [4]:
def check_image_validity(file_path):
    try:
        img = tf.io.read_file(file_path)
        tf.image.decode_image(img)
        return True
    except:
        return False
training_set=tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/dataset/mango/train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,

    verbose=True,
)

Found 2798 files belonging to 8 classes.


###validation image processing

In [ ]:
def check_image_validity(file_path):
    try:
        img = tf.io.read_file(file_path)
        tf.image.decode_image(img)
        return True
    except:
        return False
validation_set=tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/dataset/mango/validation',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,

    verbose=True,
)

In [ ]:
for x,y in training_set:
  print(x, x.shape)
  print(y, y.shape)
  break

##Building Model

In [5]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential


##Building convulation Layer

In [10]:
model=Sequential()
# input layer
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(256,256,3)))
model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu' ))
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu' ))
model.add(Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))


model.add(Dropout(0.25))
#flattening the input layer
model.add(Flatten())
#hidden layer
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.4))
#output layer
model.add(Dense(units=8, activation='softmax'))



##Compiling model

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])


In [12]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 254, 254, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 127, 127, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 125, 125, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 62, 62, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 60, 60, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 30, 30, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 14, 14, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     4,718,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,433,128 (35.98 MB)

 Trainable params: 9,433,128 (35.98 MB)

 Non-trainable params: 0 (0.00 B)

###Training Model

##Model Evaluation

##Saving Model